In [12]:
from Strategies.StrategyBB import StrategyBB
from Strategies.StrategyMACD import StrategyMACD
from Strategies.StrategyOBV import StrategyOBV
from Strategies.StrategyRSI import StrategyRSI
from Strategies.StrategyST import StrategyST
from historyData.HistoryData import HistoryData
from tinkoff.invest.grpc.marketdata_pb2 import CandleInterval
from datetime import datetime

import asyncio

In [13]:
candle_interval = CandleInterval.CANDLE_INTERVAL_1_MIN
data_set = HistoryData()

In [14]:
from time import sleep

candles_down = None
candles_up = None
candles_flat = None
candles_short_up = None
candles_short_down = None

async def data_get():
    global candles_down, candles_up, candles_flat, candles_short_up, candles_short_down
    
    candles_short_up = await (data_set.get_tinkoff_server_data(datetime(2023, 1, 1), datetime(2023, 2, 1), candle_interval))
    candles_short_down = await (data_set.get_tinkoff_server_data(datetime(2021, 1, 1), datetime(2021, 2, 1), candle_interval))
    candles_down = await (data_set.get_tinkoff_server_data(datetime(2022, 2, 1), datetime(2022, 3, 1), candle_interval))
    candles_up = await (data_set.get_tinkoff_server_data(datetime(2023, 3, 1), datetime(2023, 4, 1), candle_interval))
    candles_flat = await (data_set.get_tinkoff_server_data(datetime(2024, 1, 1), datetime(2024, 2, 1), candle_interval))

# Run the event loop directly using await
await data_get()

# Creates a Data Set For Future Model

In [15]:
all_candles = [candles_down, candles_up, candles_flat, candles_short_up, candles_short_down]
len(all_candles[3])

21830

In [21]:
from AppLogic.Settings import Settings
from tinkoff.invest.utils import quotation_to_decimal

commission = 0.0005
all_data_index_buy_sell = []

for candles in all_candles:
    data_index_buy_sell = []
    total_earn = 0
    total_sum_earned = 0
    bought_at = -1
    for i, candle in enumerate(list(candles)):
        bought_at = float(quotation_to_decimal(candle.close))
        this_candle_earned = 0
        for j in range(i + 1, len(candles)):
            further_candle_price = float(quotation_to_decimal(candles[j].close))
            this_candle_earned = bought_at - further_candle_price
            if this_candle_earned > 0.5 / 100 * bought_at:  # 0.1 is stop loss
                data_index_buy_sell.append({"bought": i})
                break
            elif this_candle_earned < -0.05 / 100 * bought_at:  # 0.05 is stop loss
                break
            
    all_data_index_buy_sell.append(data_index_buy_sell)
    print(f"Length of the marked data: {len(data_index_buy_sell)}", f"total length: {len(candles)}")

Length of the marked data: 3444 total length: 17378
Length of the marked data: 4348 total length: 22663
Length of the marked data: 1416 total length: 21769
Length of the marked data: 3925 total length: 21830
Length of the marked data: 2317 total length: 15422


In [22]:
import pandas as pd

all_new_data_buy = list()
for data_index_buy_sell in all_data_index_buy_sell:
    new_data_buy = list()
    for i, data in enumerate(data_index_buy_sell):
        if list(data_index_buy_sell[i].keys())[0] != "sold":
            new_data_buy.append(data)
    all_new_data_buy.append(new_data_buy)

all_df_data = list()

for c_i, candles in enumerate(all_candles):
    df_data = {"index": [], "result": [], "close": [],
               "open": [], "high": [], "low": [],
               "volume": []}
    buy_counter = 0
    for i, candle in enumerate(list(candles)):
        df_data["index"].append(i)
        df_data["close"].append(float(quotation_to_decimal(candle.close)))
        df_data["open"].append(float(quotation_to_decimal(candle.open)))
        df_data["high"].append(float(quotation_to_decimal(candle.high)))
        df_data["low"].append(float(quotation_to_decimal(candle.low)))
        df_data["volume"].append(candle.volume)
        
        if i == all_new_data_buy[c_i][buy_counter]["bought"]:
            df_data["result"].append(1)
            buy_counter += 1 if buy_counter < len(all_new_data_buy[c_i]) - 1 else 0
        else:
            df_data["result"].append(0)
    all_df_data.append(df_data)
        
print(f"All different period data len: {len(all_df_data)}", f"true: {all_df_data[2]['result'].count(1)}", f"false: {all_df_data[2]['result'].count(0)}")

All different period data len: 5 true: 1416 false: 20353


# Add Strategy Signals To DF

In [23]:
from Strategies.StrategyAroon import StrategyAroon
from Strategies.StrategyStochRSI import StrategyStochRSI
from Strategies.StrategyEMA import StrategyEMA

# candle interval init (for this case is always 1 min)
candle_interval = CandleInterval.CANDLE_INTERVAL_1_MIN

# init all strategies
EMA_down = StrategyEMA()
EMA_up = StrategyEMA()
EMA_flat = StrategyEMA()
EMA_short_up = StrategyEMA()
EMA_short_down = StrategyEMA()

RSI_down = StrategyRSI()
RSI_up = StrategyRSI()
RSI_flat = StrategyRSI()
RSI_short_up = StrategyRSI()
RSI_short_down = StrategyRSI()

MACD_down = StrategyMACD()
MACD_up = StrategyMACD()
MACD_flat = StrategyMACD()
MACD_short_up = StrategyMACD()
MACD_short_down = StrategyMACD()

BB_down = StrategyBB()
BB_up = StrategyBB()
BB_flat = StrategyBB()
BB_short_up = StrategyBB()
BB_short_down = StrategyBB()

ST_down = StrategyST()
ST_up = StrategyST()
ST_flat = StrategyST()
ST_short_up = StrategyST()
ST_short_down = StrategyST()

StochRSI_down = StrategyStochRSI()
StochRSI_up = StrategyStochRSI()
StochRSI_flat = StrategyStochRSI()
StochRSI_short_up = StrategyStochRSI()
StochRSI_short_down = StrategyStochRSI()

Aroon_down = StrategyAroon()
Aroon_up = StrategyAroon()
Aroon_flat = StrategyAroon()
Aroon_short_up = StrategyAroon()
Aroon_short_down = StrategyAroon()

OBV_down = StrategyOBV()
OBV_up = StrategyOBV()
OBV_flat = StrategyOBV()
OBV_short_up = StrategyOBV()
OBV_short_down = StrategyOBV()

strategies_blocks = [[EMA_down, EMA_up, EMA_flat, EMA_short_up, EMA_short_down], 
                    [RSI_down, RSI_up, RSI_flat, RSI_short_up, RSI_short_down], 
                    [MACD_down, MACD_up, MACD_flat, MACD_short_up, MACD_short_down], 
                    [BB_down, BB_up, BB_flat, BB_short_up, BB_short_down],
                    [StochRSI_down, StochRSI_up, StochRSI_flat, StochRSI_short_up, StochRSI_short_down],
                    [ST_down, ST_up, ST_flat, ST_short_up, ST_short_down],
                    [Aroon_down, Aroon_up, Aroon_flat, Aroon_short_up, Aroon_short_down],
                    [OBV_down, OBV_up, OBV_flat, OBV_short_up, OBV_short_down]]

for strategy_block in strategies_blocks:
    strategy_block[0].initialize_moving_avg_container(candles_down[:strategy_block[0].history_candles_length])
    strategy_block[1].initialize_moving_avg_container(candles_up[:strategy_block[1].history_candles_length])
    strategy_block[2].initialize_moving_avg_container(candles_flat[:strategy_block[2].history_candles_length])
    strategy_block[3].initialize_moving_avg_container(candles_flat[:strategy_block[3].history_candles_length])
    strategy_block[4].initialize_moving_avg_container(candles_flat[:strategy_block[4].history_candles_length])

print(f"Strategy blocks len: {len(strategies_blocks)}")     

Strategy blocks len: 8


In [24]:
all_df_data_st = [{} for i in range(len(strategies_blocks[0]))]

for strategy_block in strategies_blocks:
    for str_i, strategy_type in enumerate(strategy_block):
        for candle_index in range(strategy_type.history_candles_length, len(all_candles[str_i])):
            params = strategy_type.get_candle_param(all_candles[str_i][candle_index])
            for param_name in params:
                if candle_index == strategy_type.history_candles_length:
                    all_df_data_st[str_i][param_name] = list()
                    [all_df_data_st[str_i][param_name].append(None) for i in range(strategy_type.history_candles_length)]
                all_df_data_st[str_i][param_name].append(params[param_name])
            
print(f"Small length check should be the length of candles: {len(all_df_data_st[2]['BB_lower'])}, candle len: {len(all_candles[2])}")

Small length check should be the length of candles: 21769, candle len: 21769


In [25]:
# convert to df
for i in range(len(all_df_data)):
    all_df_data[i] = pd.DataFrame(all_df_data[i])

for i in range(len(all_df_data)):
    for name in all_df_data_st[i]:
        all_df_data[i][name] = all_df_data_st[i][name]

biggest_gap = 0
for strategy_block in strategies_blocks:
    for strategy_type in strategy_block:
        if strategy_type.history_candles_length > biggest_gap:
            biggest_gap = strategy_type.history_candles_length

# delete first NaN elements
for i, df in enumerate(all_df_data):
    all_df_data[i] = df.drop(df.index[:biggest_gap])

concatenated_df = pd.DataFrame()
for i, data in enumerate(all_df_data):
    data.to_csv(f"./marked_data/result_with_parameters_{i}.csv", index=False)
    
all_df_data[0]

,index,result,close,open,high,low,volume,EMA_prev_diff,EMA_current_diff,prev_RSI,...,BB_lower,StochRSI_K,StochRSI_D,ST_upper,ST_lower,ART,aroon_up,aroon_low,OBV,OBV_MA
200,200,1,265.12,265.00,265.34,264.62,65474,3.245200,3.409157,47.078772,...,263.052341,2.558635,31.683117,272.525571,268.662816,1.905844,28.0,92.0,-225535.0,-1.425849e+05
201,201,1,264.22,265.11,265.23,264.14,83902,3.409157,3.629551,47.195712,...,262.786101,0.000000,30.830239,272.525571,268.662816,1.918121,24.0,100.0,-309437.0,-1.649709e+05
202,202,0,262.86,264.22,264.48,262.53,200461,3.629551,3.939523,46.418695,...,262.514976,0.000000,30.830239,272.415738,268.662816,1.986913,20.0,100.0,-509898.0,-2.050541e+05
203,203,0,263.69,262.86,263.85,262.82,77133,3.939523,4.142661,45.280853,...,262.442282,17.887931,36.792883,271.724355,268.662816,1.989785,16.0,96.0,-432765.0,-2.365421e+05
204,204,0,264.50,263.69,264.74,263.58,71947,4.142661,4.251615,46.095365,...,262.511790,35.344828,42.611848,270.683398,268.662816,2.001133,12.0,92.0,-360818.0,-2.509723e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17373,17373,0,130.70,130.75,130.87,130.50,13611,-0.975887,-1.009690,53.147416,...,129.736794,41.176471,44.555729,109.181222,276.175918,1.557147,16.0,28.0,-25842115.0,-2.586729e+07
17374,17374,0,130.80,130.70,130.98,130.61,17154,-1.009690,-1.047542,53.147416,...,129.715555,44.444444,45.645054,109.181222,276.175918,1.544670,12.0,24.0,-25824961.0,-2.586588e+07
17375,17375,0,130.95,130.85,131.00,130.80,11660,-1.047542,-1.093248,53.347018,...,129.837038,48.717949,47.069555,109.181222,276.175918,1.521692,100.0,20.0,-25813301.0,-2.586336e+07
17376,17376,0,131.30,130.85,131.50,130.74,37603,-1.093248,-1.163004,53.646210,...,129.737552,56.521739,49.670819,109.181222,276.175918,1.537579,100.0,16.0,-25775698.0,-2.585746e+07
